In [1]:
import numpy
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)
import matplotlib as plt
plt.rcParams["figure.figsize"] = (10, 5)
import sys, glob
import urllib3
import h5py
import cdsapi, zipfile, os, time
import warnings
import shutil
import xarray
from datetime import date
warnings.filterwarnings('ignore')
# import pycountry
sys.path.append(os.getcwd()+'/../cds-backend/code/')
import cds_eua3 as eua
import numba
import copy
import glob
from numba import njit
import pandas
import glob
import pickle
import multiprocessing
from functools import partial

/tmp/ipykernel_1061719/832296286.py:7: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


## 1 station 

### csv

In [3]:
tt = time.time()
eua.vm_request_wrapper({'variable': 'temperature', 'statid': ['11035'], 'fast_csv':'True', 'speed_test':1}, vm_url='http://localhost:8004',overwrite=True)
print(time.time() - tt)

23.11182403564453


### nc

In [4]:
tt = time.time()
eua.vm_request_wrapper({'variable': 'temperature', 'statid': ['11035'], 'speed_test':1}, vm_url='http://localhost:8004',overwrite=True)
print(time.time() - tt)

7.497774124145508


## 1 month all stations

### csv

In [5]:
tt = time.time()
eua.vm_request_wrapper({'variable': 'temperature', 'date':['20100101','20100201'], 'fast_csv':'True', 'speed_test':1}, vm_url='http://localhost:8004',overwrite=True)
print(time.time() - tt)

3.774242639541626


### nc

In [6]:
tt = time.time()
eua.vm_request_wrapper({'variable': 'temperature', 'date':['20100101','20100201'], 'speed_test':1}, vm_url='http://localhost:8004',overwrite=True)
print(time.time() - tt)

4.337301969528198


## 300 hPa all stations

### csv

In [7]:
tt = time.time()
eua.vm_request_wrapper({'variable': 'temperature', 'pressure_level':[30000], 'fast_csv':'True', 'speed_test':1}, vm_url='http://localhost:8004',overwrite=True)
print(time.time() - tt)

189.64754700660706


### nc

In [8]:
tt = time.time()
eua.vm_request_wrapper({'variable': 'temperature', 'pressure_level':[30000], 'speed_test':1}, vm_url='http://localhost:8004',overwrite=True)
print(time.time() - tt)

114.38222670555115
